In [2]:
import h5py
import numpy as np

FILEPATH = "/workspace/velocityHistory.h5"
with h5py.File(FILEPATH, "r") as h5f:
    rey_100 = h5f["Reynolds_100.0"][:].T
    rey_400 = h5f["Reynolds_400.0"][:].T
    rey_combined = np.concatenate([rey_100, rey_400], axis=0)
    display(rey_combined.shape)

(6000, 65, 65, 2)

In [19]:
rey_combined = rey_combined[:, 1:-1, 1:-1, :]
display(rey_combined.shape)

(6000, 63, 63, 2)

In [20]:
import keras
from keras import layers
from keras.optimizers import Adam

original_dim = 63 * 63 * 2
L1_dim = 31 * 31 * 2
L2_dim = 15 * 15 * 2
encoding_dim = 8 * 8 * 2

input_vf = keras.Input(shape=(original_dim,))
encoded = layers.Dense(L1_dim, activation='relu')(input_vf)
encoded = layers.Dense(L2_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

decoded = layers.Dense(L2_dim, activation='relu')(encoded)
decoded = layers.Dense(L1_dim, activation='relu')(decoded)
decoded = layers.Dense(original_dim, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_vf, decoded)
opt = Adam(learning_rate=1e-4)
autoencoder.compile(optimizer=opt, loss='mse')

In [21]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 7938)]            0         
                                                                 
 dense_12 (Dense)            (None, 1922)              15258758  
                                                                 
 dense_13 (Dense)            (None, 450)               865350    
                                                                 
 dense_14 (Dense)            (None, 128)               57728     
                                                                 
 dense_15 (Dense)            (None, 450)               58050     
                                                                 
 dense_16 (Dense)            (None, 1922)              866822    
                                                                 
 dense_17 (Dense)            (None, 7938)              1526

In [22]:
x_train = rey_combined
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))

autoencoder.fit(x_train, x_train,
       epochs=100,
       batch_size=32,
       shuffle=True)

Epoch 1/100
188/188 [==============================] - 3s 7ms/step - loss: 0.0363
Epoch 2/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0137
Epoch 3/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0133
Epoch 4/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0131
Epoch 5/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0131
Epoch 6/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0131
Epoch 7/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0131
Epoch 8/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0131
Epoch 9/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0130
Epoch 10/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0130
Epoch 11/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0130
Epoch 12/100
188/188 [==============================] - 1s 7ms/step - loss: 0.0130
Epoch 13/100


In [35]:
autoencoder.save("/workspace/autoencoder.hdf5")

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
from keras.models import Model

# Extract the encoder part (7938 -> 1922 -> 450 -> 128)
encoder_input = autoencoder.input
encoder_output = autoencoder.layers[2].output  # Index 2 is the last layer of the encoder
encoder_model = Model(inputs=encoder_input, outputs=encoder_output)

# Extract the decoder part (128 -> 450 -> 1922 -> 7938)
decoder_input = autoencoder.layers[3].input  # Index 3 is the first layer of the decoder
decoder_output = autoencoder.layers[-1].output  # Index -1 is the last layer of the decoder
decoder_model = Model(inputs=decoder_input, outputs=decoder_output)

In [43]:
predicted = decoder_model.predict(encoder_model.predict(x_sample))
display(np.mean(predicted - x_sample))

1/1 [==============================] - 0s 14ms/step


0.00285219507599595